In [13]:
import random
from sqlalchemy.orm import sessionmaker
from models import User, engine

In [2]:
# Create session
Session = sessionmaker(bind=engine)
session = Session()

### CRUD Operations

In [3]:
user1 = User(name="Amit", age=25)
user2 = User(name="Mobin", age=27)
user3 = User(name="Santu", age=28)
user4 = User(name="Morgan", age=40)

In [4]:
## CREATE 
session.add(user1)
session.add_all([user2, user3, user4])
session.commit()

In [6]:
## READ 
users = session.query(User).all()
for user in users:
    print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 1, Name : Amit, Age : 25
ID : 2, Name : Mobin, Age : 27
ID : 3, Name : Santu, Age : 28
ID : 4, Name : Morgan, Age : 40


In [9]:
## UPDATE
user = session.query(User).filter_by(name = "Amit").one_or_none()
user.name = "Farooqe"
print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 1, Name : Farooqe, Age : 25


In [11]:
## DELETE 
user = session.query(User).filter_by(id=4).one_or_none()
session.delete(user)
session.commit()

### Data Manipulation

In [14]:
## Add some new data 

names = ["Thor", "Hulk", "Goku", "Vegeta", "Morgoth", "Frodo", "Pippin"]
ages = [20, 21, 22, 23, 24, 25, 26, 26, 27, 28, 29, 30]

for i in range(len(names)):
    user = User(name=names[i], age=random.choice(ages))
    session.add(user)
session.commit()

### Ordering data

In [19]:
users = session.query(User).order_by(User.age).all()
for user in users:
    print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 6, Name : Goku, Age : 20
ID : 8, Name : Morgoth, Age : 23
ID : 9, Name : Frodo, Age : 23
ID : 10, Name : Pippin, Age : 24
ID : 1, Name : Farooqe, Age : 25
ID : 4, Name : Thor, Age : 25
ID : 5, Name : Hulk, Age : 26
ID : 2, Name : Mobin, Age : 27
ID : 3, Name : Santu, Age : 28
ID : 7, Name : Vegeta, Age : 28


### Filtering Data

In [21]:
users = session.query(User).all() 
filtered_users = session.query(User).filter(User.age >= 25).all() 

print(f"Total users : {len(users)}")
print(f"Filtered users : {len(filtered_users)}")

Total users : 10
Filtered users : 6
